In [58]:
import pandas as pd
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [59]:
total_crimes_by_type = pd.read_csv('crime_by_tipe.csv')
total_crimes_by_type

,Unnamed: 0,Crime Type,Year,Total_Crimes
0,0,ASSAULT,2014,16515.0
1,1,ASSAULT,2015,17858.0
2,2,ASSAULT,2016,18608.0
3,3,ASSAULT,2017,18906.0
4,4,ASSAULT,2018,19565.0
...,...,...,...,...
94,94,THEFTOVER,2020,1209.0
95,95,THEFTOVER,2021,1052.0
96,96,THEFTOVER,2022,1444.0
97,97,THEFTOVER,2023,1719.0


In [60]:
total_crimes_by_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    99 non-null     int64  
 1   Crime Type    99 non-null     object 
 2   Year          99 non-null     int64  
 3   Total_Crimes  99 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.2+ KB


In [61]:
total_crimes_by_type['Crime Type'] = total_crimes_by_type['Crime Type'].astype('string')
total_crimes_by_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    99 non-null     int64  
 1   Crime Type    99 non-null     string 
 2   Year          99 non-null     int64  
 3   Total_Crimes  99 non-null     float64
dtypes: float64(1), int64(2), string(1)
memory usage: 3.2 KB


In [62]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_indices = y_true != 0
    return np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100

all_predictions = pd.DataFrame()
total_maes = []
total_rmses = []
total_mapes = []
crime_types = total_crimes_by_type['Crime Type'].unique()

for crime in crime_types:
    crime_type_data = total_crimes_by_type[total_crimes_by_type['Crime Type'] == crime].copy()
    crime_type_data = crime_type_data.set_index('Year')
    crime_type_data.index = crime_type_data.index.astype(int)
    total_crimes_series = crime_type_data['Total_Crimes']
    
    train_data = total_crimes_series.loc[total_crimes_series.index <= 2023]
    test_data = total_crimes_series.loc[total_crimes_series.index > 2023]

    auto_model = pm.auto_arima(train_data,
                           start_p=1, start_q=1,
                           max_p=10, max_q=10,
                           m=1, 
                           seasonal=False,
                           d=None, 
                           trace=True,
                           error_action='ignore',
                           suppress_warnings=True,
                           stepwise=True)
    best_order = auto_model.order

    model = ARIMA(train_data, order=best_order)
    fitted_model = model.fit()
    start_year = test_data.index[0]
    end_year = test_data.index[-1]

    predictions = fitted_model.predict(start=start_year, end=end_year)
    predictions_df = predictions.to_frame(name='Predicted_Crimes')
    predictions_df['Crime_Type'] = crime

    mae = mean_absolute_error(test_data, predictions)
    rmse = np.sqrt(mean_squared_error(test_data, predictions))
    mape = mean_absolute_percentage_error(test_data, predictions)

    total_maes.append(mae)
    total_rmses.append(rmse)
    total_mapes.append(mape)

    evaluation_df = predictions.to_frame(name='Predicted_Crimes')
    evaluation_df['Crime Type'] = crime
    evaluation_df['Actual_Crimes'] = test_data 
    evaluation_df['MAE'] = mae
    evaluation_df['RMSE'] = rmse
    evaluation_df['MAPE'] = mape

    all_predictions = pd.concat([all_predictions, evaluation_df])
    
    if total_maes:
        total_mae = np.mean(total_maes)
        total_rmse = np.mean(total_rmses)
        total_mape = np.mean(total_mapes)
        print(all_predictions)
        print(f"\nMean Absolute Error (MAE): {total_mae:.2f}")
        print(f"Root Mean Squared Error (RMSE): {total_rmse:.2f}")
        print(f"Mean Absolute Percentage Error (MAPE): {total_mape:.2f}%")
    else:
        print("\nNo models were successfully fitted and evaluated.")

    print(all_predictions)

    

Performing stepwise search to minimize aic


/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,0,1)(0,0,0)[0]             : AIC=187.811, Time=0.12 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=227.975, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=inf, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=224.862, Time=0.02 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=inf, Time=0.29 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=189.631, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=225.320, Time=0.02 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=inf, Time=0.06 sec


/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=0.16 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=185.250, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=184.412, Time=0.08 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=184.752, Time=0.02 sec


...

/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,0,0)(0,0,0)[0]             : AIC=142.792, Time=0.06 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=0.15 sec


/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.15 sec

Best model:  ARIMA(1,0,1)(0,0,0)[0]          
Total fit time: 0.772 seconds
      Predicted_Crimes   Crime Type  Actual_Crimes          MAE         RMSE  \
2024      19425.700540      ASSAULT        25445.0  6019.299460  6019.299460   
2024       7907.995601    AUTOTHEFT         9256.0  1348.004399  1348.004399   
2024       3456.609721    BIKETHEFT         2709.0   747.609721   747.609721   
2024       7000.755621   BREAKENTER         6842.0   158.755621   158.755621   
2024         73.199995     HOMICIDE           85.0    11.800005    11.800005   
2024       3285.874177      ROBBERY         3156.0   129.874177   129.874177   
2024        342.416196     SHOOTING          460.0   117.583804   117.583804   
2024       8925.800000  THEFTFROMMV         7644.0  1281.800000  1281.800000   
2024       1235.857650    THEFTOVER         1896.0   660.142350   660.142350   

           MAPE  
2024  23.656119  
2024  14.563574  
2024  2

/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive pa

In [63]:
total_mae, total_rmse, total_mape

(1163.8743929705875, 1163.8743929705875, 18.142534393352317)

In [64]:
# prediction for 2025:

predictions_2025 = pd.DataFrame()
for crime in crime_types:
    crime_type_data = total_crimes_by_type[total_crimes_by_type['Crime Type'] == crime].copy()
    
    if 'Year' not in crime_type_data.columns:
        total_crimes_by_type = total_crimes_by_type.reset_index()
    
    crime_type_data['Year'] = pd.to_datetime(crime_type_data['Year'], format='%Y')
    crime_type_data = crime_type_data.set_index('Year')
    crime_type_data.index = crime_type_data.index.astype(int)
    total_crimes_series = crime_type_data['Total_Crimes']
    train_data = total_crimes_series
    
    auto_model = pm.auto_arima(train_data,
                           start_p=1, start_q=1,
                           max_p=12, max_q=12,
                           m=1, 
                           seasonal=False,
                           d=None, 
                           trace=True,
                           error_action='ignore',
                           suppress_warnings=True,
                           stepwise=True,
                           trend = 't')
    best_order = auto_model.order
    
    model = ARIMA(train_data, order=best_order)
    fitted_model = model.fit()
    predictions = fitted_model.forecast(steps=1)

    predictions_df = pd.DataFrame({
            'Crime Type': [crime],
            'Year': [2025],
            'Predicted_Crimes': [predictions.iloc[0]]
        })
    predictions_2025 = pd.concat([predictions_2025, predictions_df], ignore_index=True)
        
predictions_2025


/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=inf, Time=0.18 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=234.167, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=198.624, Time=0.10 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=inf, Time=0.05 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=200.604, Time=0.09 sec


/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,0,1)(0,0,0)[0]             : AIC=202.580, Time=0.20 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=198.624, Time=0.06 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 0.708 seconds
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=inf, Time=0.10 sec


/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the su

...

 ARIMA(2,0,0)(0,0,0)[0]             : AIC=151.580, Time=0.11 sec


/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,0,1)(0,0,0)[0]             : AIC=inf, Time=0.81 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=149.582, Time=0.07 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 1.738 seconds


/opt/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the su

,Crime Type,Year,Predicted_Crimes
0,ASSAULT,2025,24797.071408
1,AUTOTHEFT,2025,6115.172173
2,BIKETHEFT,2025,2840.847033
3,BREAKENTER,2025,6929.497823
4,HOMICIDE,2025,74.538275
5,ROBBERY,2025,3214.423409
6,SHOOTING,2025,514.678439
7,THEFTFROMMV,2025,8809.272670
8,THEFTOVER,2025,1820.642861


In [65]:
predictions_2025['Predicted_Crimes'] = predictions_2025['Predicted_Crimes'].astype(int)
predictions_2025

,Crime Type,Year,Predicted_Crimes
0,ASSAULT,2025,24797
1,AUTOTHEFT,2025,6115
2,BIKETHEFT,2025,2840
3,BREAKENTER,2025,6929
4,HOMICIDE,2025,74
5,ROBBERY,2025,3214
6,SHOOTING,2025,514
7,THEFTFROMMV,2025,8809
8,THEFTOVER,2025,1820
